<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=e5bd874f1d10fa402509230ecfd39e2c25375497c2a1a34b694acfbdeddd7fcb
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-08 12:25:38
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.28 C
CY Investment:  1.55 C
Reserve:  4.95 L
Current:  1.40 C
-------------------
Today PnL: -1.65 L (-1.17%)
Current PnL: -28.66 L (-18.51%)
CY Booked + Current PnL: -14.37 L (-9.28%)
-------------------
Total profit:  1.36 L
Total loss:  -30.02 L
-------------------
Total Booked + Current PnL: 12.61 L (9.87%)
Total Booked PnL: 41.27 L (32.29%)
Curr Year Booked PnL: 14.29 L (10.24%)
Prev Year Booked PnL: 26.98 L (21.11%)
Est FTT:  2.33 C
Est FTT PnL: 93.29 L (66.82%)
Deployed:  1.28 C
Current:  1.40 C
CAGR/XIRR %: 4.97%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,-0.23,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.38,253.0,-0.99,0.62,10.15,OX40N,NTT,DURABLES
43,ITC,452.00,-0.01,-1.07,11.63,10.44,23027.0,-2142.0,197996.0,-40.74,49.0,X-LC,1.57,5.0,-0.09,1.47,4.43,X40,NTT,FMCG
39,INDIGOPNTS,1408.00,-1.15,-13.24,15.29,0.02,23155.0,-23118.0,151441.0,131.03,50.0,M-SC,16.83,234.0,-1.00,1.12,31.73,OX40N,NTT,PAINTS
50,MASFIN,398.61,-1.51,-3.32,26.24,22.05,24856.0,-3255.0,94725.0,-16.44,53.0,H-SC,6.87,164.0,-0.13,0.70,37.69,XR,ATH,FINANCE
35,ICICIGI,2252.93,-0.62,4.68,14.82,20.19,27043.0,8155.0,182475.0,-17.91,43.0,X-MC,6.16,68.0,0.30,1.36,20.50,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,STARHEALTH,761.00,0.72,-10.76,65.02,47.26,163113.0,-30257.0,250866.0,21.68,34.0,H-SC,13.76,171.0,-0.19,1.86,34.41,XY24,NTT,INSURANCE
51,MEDANTA,1486.00,0.58,-2.29,26.99,24.08,33479.0,-2909.0,124041.0,-10.14,35.0,X-SC,5.84,89.0,-0.09,0.92,16.88,XY24,NTT,HEALTHCARE
32,HCLTECH,1922.01,0.53,9.80,13.59,24.73,36101.0,23720.0,265644.0,14.46,72.0,X-LC,5.56,7.0,0.66,1.97,26.53,X40,ATH,IT
84,WIPRO,420.00,0.38,7.01,60.99,72.27,110586.0,11871.0,181319.0,-8.35,73.0,M-LC,2.80,99.0,0.11,1.35,13.61,XR,NTT,IT
5,ASIANPAINT,3460.25,0.38,3.76,16.57,20.95,43285.0,9460.0,261228.0,2.74,80.0,X-LC,18.48,31.0,0.22,1.94,40.73,X40,ATH,PAINTS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,KPIGREEN,730.66,-5.37,-19.39,82.30,46.95,83125.0,-24295.0,101002.0,-25.58,27.0,H-SC,4.22,154.0,-0.29,0.75,26.53,MH,ATH,POWER
18,COFFEEDAY,80.00,-3.81,-45.77,149.45,35.27,92022.0,-51975.0,61574.0,-57.75,21.0,L-SC,12.92,270.0,-0.56,0.46,50.70,XR,NTT,HOTELS
57,RAJOOENG,143.10,-3.04,-34.03,111.62,39.61,75477.0,-34880.0,67620.0,-55.45,27.0,H-SC,25.14,135.0,-0.46,0.50,0.00,AR,ATH,MISC
15,CAMS,4762.00,-3.04,-80.29,533.16,24.79,264655.0,-202217.0,49639.0,-81.09,41.0,X-SC,2.09,86.0,-0.76,0.37,21.35,X40N,NTT,MISC
28,GREENPANEL,537.00,-3.03,-38.14,131.41,43.14,148815.0,-69833.0,113245.0,153.88,24.0,M-SC,8.18,240.0,-0.47,0.84,6.52,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-1.91,-8.50,127.22,107.91,171872.0,-12546.0,135098.0,-25.42,26.0,M-SC,11.59,216.0,-0.07,1.00,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-1.15,-13.24,15.29,0.02,23155.0,-23118.0,151441.0,131.03,50.0,M-SC,16.83,234.0,-1.00,1.12,31.73,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-0.23,-16.81,20.31,0.09,17028.0,-16937.0,83840.0,94.66,37.0,M-SC,2.38,253.0,-0.99,0.62,10.15,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.32,-24.88,51.05,13.47,103418.0,-67086.0,202581.0,-69.09,28.0,H-SC,2.86,158.0,-0.65,1.50,3.01,XY24,NTT,PAINTS
66,SIS,528.00,-1.29,-25.41,62.56,21.26,51815.0,-28208.0,82824.0,1952.33,43.0,H-SC,4.24,166.0,-0.54,0.62,11.90,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.0,0.38,7.01,60.99,72.27,110586.0,11871.0,181319.0,-8.35,73.0,M-LC,2.8,99.0,0.11,1.35,13.61,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.38,7.01,60.99,72.27,110586.0,11871.0,181319.0,-8.35,73.0,M-LC,2.80,99.0,0.11,1.35,13.61,XR,NTT,IT
38,INDIAMART,4810.62,-2.29,-2.90,112.91,106.72,135214.0,-3582.0,119754.0,-51.96,33.0,H-SC,1.31,139.0,-0.03,0.89,19.79,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.26,-2.03,38.44,35.63,78925.0,-4254.0,205320.0,-15.85,39.0,H-MC,3.17,119.0,-0.05,1.52,14.30,AR,ATH,PHARMA
37,IEX,219.00,-1.82,-3.18,53.51,48.63,103590.0,-6364.0,193590.0,-35.53,50.0,H-SC,14.12,136.0,-0.06,1.44,8.00,XR,NTT,MISC
25,FINCABLES,1641.55,-1.91,-8.50,127.22,107.91,171872.0,-12546.0,135098.0,-25.42,26.0,M-SC,11.59,216.0,-0.07,1.00,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-1.85,-23.29,93.54,48.45,143864.0,-46707.0,153799.0,-28.89,10.0,X-MC,10.64,64.0,-0.32,1.14,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.65,-38.66,120.41,35.20,95038.0,-49741.0,78929.0,3.54,23.0,X-SC,15.03,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.23,-11.90,38.01,21.58,46928.0,-16677.0,123462.0,-55.55,26.0,X-MC,5.33,56.0,-0.36,0.92,9.30,X40N,ATH,IT
46,JSWINFRA,342.00,-1.88,-10.99,29.45,15.23,52520.0,-22012.0,178335.0,-23.85,27.0,X-MC,7.54,66.0,-0.42,1.32,17.72,X40N,NTT,REALTY
54,PGHH,17907.65,-0.35,-6.31,42.77,33.76,80470.0,-12675.0,188145.0,-33.85,28.0,X-MC,5.07,57.0,-0.16,1.40,0.00,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.0,-0.93,-20.74,93.65,53.50,223992.0,-62571.0,239180.0,-61.59,34.0,X-MC,1.36,58.0,-0.28,1.78,6.24,XY24,NTT,FMCG
43,ITC,452.0,-0.01,-1.07,11.63,10.44,23027.0,-2142.0,197996.0,-40.74,49.0,X-LC,1.57,5.0,-0.09,1.47,4.43,X40,NTT,FMCG
3,ACC,3906.0,0.17,-24.30,116.86,64.15,210488.0,-57831.0,180120.0,-55.88,39.0,X-SC,1.67,82.0,-0.27,1.34,1.04,XY24,BTT,CEMENT
15,CAMS,4762.0,-3.04,-80.29,533.16,24.79,264655.0,-202217.0,49639.0,-81.09,41.0,X-SC,2.09,86.0,-0.76,0.37,21.35,X40N,NTT,MISC
36,ICICIPRULI,790.0,0.26,4.05,25.87,30.97,47575.0,7163.0,183901.0,-20.46,59.0,X-MC,2.21,75.0,0.15,1.37,17.12,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.65,-38.66,120.41,35.20,95038.0,-49741.0,78929.0,3.54,23.0,X-SC,15.03,92.0,-0.52,0.59,0.0,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-1.85,-23.29,93.54,48.45,143864.0,-46707.0,153799.0,-28.89,10.0,X-MC,10.64,64.0,-0.32,1.14,0.0,X40N,ATH,FINANCE
55,QUESS,424.00,-1.30,-32.89,112.84,42.84,49229.0,-21379.0,43627.0,-55.92,32.0,X-SC,37.32,83.0,-0.43,0.32,0.0,XY24,NTT,MISC
53,PAGEIND,51605.08,-0.91,-9.33,39.04,26.07,57959.0,-15280.0,148460.0,-33.27,30.0,X-MC,11.19,55.0,-0.26,1.10,0.0,X40,ATH,APPARELS
54,PGHH,17907.65,-0.35,-6.31,42.77,33.76,80470.0,-12675.0,188145.0,-33.85,28.0,X-MC,5.07,57.0,-0.16,1.40,0.0,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,-1.30,-32.89,112.84,42.84,49229.0,-21379.0,43627.0,-55.92,32.0,X-SC,37.32,83.0,-0.43,0.32,0.00,XY24,NTT,MISC
15,CAMS,4762.00,-3.04,-80.29,533.16,24.79,264655.0,-202217.0,49639.0,-81.09,41.0,X-SC,2.09,86.0,-0.76,0.37,21.35,X40N,NTT,MISC
59,RELAXO,1176.00,-1.16,-48.22,194.15,52.31,145927.0,-69998.0,75162.0,-44.83,32.0,X-SC,5.75,91.0,-0.48,0.56,0.61,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.65,-38.66,120.41,35.20,95038.0,-49741.0,78929.0,3.54,23.0,X-SC,15.03,92.0,-0.52,0.59,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.23,-11.90,38.01,21.58,46928.0,-16677.0,123462.0,-55.55,26.0,X-MC,5.33,56.0,-0.36,0.92,9.30,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,0.32,-11.74,35.14,19.28,107300.0,-40606.0,305350.0,-22.98,71.0,X-LC,6.04,1.0,-0.38,2.27,12.88,X40,ATH,IT
41,INFY,2275.00,0.14,10.18,40.56,54.87,139827.0,31852.0,344740.0,-14.21,71.0,X-LC,3.40,2.0,0.23,2.56,19.31,X40,BTT,IT
75,TMPV,600.00,-0.66,-16.77,70.82,42.18,162935.0,-46341.0,230069.0,-25.52,29.0,X-LC,4.49,3.0,-0.28,1.71,0.00,XY24,NTT,AUTO
81,VBL,671.64,-2.43,-5.40,43.42,35.67,129728.0,-17067.0,298775.0,-16.71,51.0,X-LC,4.93,4.0,-0.13,2.22,7.70,X40N,ATH,FMCG
43,ITC,452.00,-0.01,-1.07,11.63,10.44,23027.0,-2142.0,197996.0,-40.74,49.0,X-LC,1.57,5.0,-0.09,1.47,4.43,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-2.67,-37.48,112.55,32.89,54328.0,-28936.0,48270.0,-696.43,55.0,L-MC,6.39,259.0,-0.53,0.36,32.93,XR,NTT,BANKS
6,ASIANTILES,137.00,-2.69,-19.44,125.51,81.67,94852.0,-18237.0,75573.0,6788.89,32.0,L-SC,15.48,271.0,-0.19,0.56,46.35,XR,NTT,CERAMICS
50,MASFIN,398.61,-1.51,-3.32,26.24,22.05,24856.0,-3255.0,94725.0,-16.44,53.0,H-SC,6.87,164.0,-0.13,0.70,37.69,XR,ATH,FINANCE
13,BSOFT,831.70,0.16,-21.42,91.88,50.79,101555.0,-30123.0,110530.0,0.89,73.0,H-SC,5.61,151.0,-0.30,0.82,28.69,XR,ATH,IT
39,INDIGOPNTS,1408.00,-1.15,-13.24,15.29,0.02,23155.0,-23118.0,151441.0,131.03,50.0,M-SC,16.83,234.0,-1.00,1.12,31.73,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,0.16,-21.42,91.88,50.79,101555.0,-30123.0,110530.0,0.89,73.0,H-SC,5.61,151.0,-0.30,0.82,28.69,XR,ATH,IT
5,ASIANPAINT,3460.25,0.38,3.76,16.57,20.95,43285.0,9460.0,261228.0,2.74,80.0,X-LC,18.48,31.0,0.22,1.94,40.73,X40,ATH,PAINTS
32,HCLTECH,1922.01,0.53,9.80,13.59,24.73,36101.0,23720.0,265644.0,14.46,72.0,X-LC,5.56,7.0,0.66,1.97,26.53,X40,ATH,IT
84,WIPRO,420.00,0.38,7.01,60.99,72.27,110586.0,11871.0,181319.0,-8.35,73.0,M-LC,2.80,99.0,0.11,1.35,13.61,XR,NTT,IT
73,TCS,4389.97,0.32,-11.74,35.14,19.28,107300.0,-40606.0,305350.0,-22.98,71.0,X-LC,6.04,1.0,-0.38,2.27,12.88,X40,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.89
1,25,45.88
2,50,76.79


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.81
MC    30.25
LC    24.95
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.30
X40      22.63
X40N     10.84
XR       10.00
XY25      9.38
AR        9.37
OX40N     7.73
SR        1.01
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.39
X-MC    23.51
X-LC    21.35
M-SC    12.08
X-SC     6.96
H-MC     4.97
M-MC     1.41
L-SC     1.38
M-LC     1.35
H-LC     1.21
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.43,-7.30,41.99
IT,12.93,-15.71,78.27
FINANCE,9.55,-20.53,71.47
ELECTRICAL,6.28,-11.20,51.93
PAINTS,6.19,-10.66,27.10
MISC,6.11,-57.84,122.51
INSURANCE,4.59,-2.42,38.52
PHARMA,4.08,-1.64,34.17
AUTO,3.45,-23.55,76.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3224937.0,21
XR,1315886.0,13
AR,1309174.0,10
X40,993921.0,14
X40N,989651.0,9
OX40N,749963.0,10
XY25,378074.0,6
SR,284287.0,2
MH,83125.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3692322.0,25
M-SC,1429748.0,15
X-MC,1319863.0,16
X-SC,984040.0,8
X-LC,887408.0,11
H-MC,408982.0,3
L-SC,275503.0,3
M-LC,110586.0,1
H-LC,70839.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1293354.0      6
           AR         897966.0      5
M-SC       XY24       811295.0      6
H-SC       XR         788737.0      7
X-SC       X40N       505816.0      3
X-MC       X40        459353.0      7
           XY24       392229.0      3
X-LC       X40        349540.0      5
H-SC       OX40N      344853.0      4
M-SC       OX40N      316481.0      5
X-SC       XY24       293196.0      3
H-SC       SR         284287.0      2
X-MC       X40N       279143.0      4
X-LC       XY24       239639.0      2
H-MC       AR         213758.0      2
X-LC       X40N       204692.0      2
H-MC       XY24       195224.0      1
X-MC       XY25       189138.0      2
L-SC       XR         186874.0      2
X-SC       X40        185028.0      2
M-SC       XR         175361.0      2
           AR         126611.0      2
M-LC       XR         110586.0      1
X-LC       XY25        93537.0      2
L-SC       OX40N       88629.0      1
H-SC       MH          83125.0      1
H-LC       AR          70839.0      1
M-MC       XY25        55689.0      1
L-MC       XR          54328.0      1
L-LC       XY25        39710.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
